In [1]:
import requests
import pandas as pd
import tensorflow as tf
import json
import base64

In [2]:
import requests
from pprint import PrettyPrinter

pp = PrettyPrinter()

response = requests.get("http://localhost:8080/v1/models/deutch-sarcastic-detection-model")


print("Raw response content:", response.text)

try:
    json_response = response.json()
    pp.pprint(json_response)
except requests.exceptions.JSONDecodeError:
    print("Response is not valid JSON")

Raw response content: {
 "model_version_status": [
  {
   "version": "1727922330",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": ""
   }
  }
 ]
}

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1727922330'}]}


In [3]:
df = pd.read_csv('headlines_dataset.csv')

In [4]:
test_data = df["headline"][10:15]

In [10]:
def prepare_json(sms):
    feature_spec = {
        "headline": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(sms, "utf-8")])),
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

endpoint = "http://localhost:8080/v1/models/deutch-sarcastic-detection-model:predict"

for text in test_data:
    json_data = prepare_json(text)
    response = requests.post(endpoint, data=json_data)
    prediction = response.json()
    if prediction:
        prediction_value = prediction['predictions'][0][0]
        result = "Sarcastic" if prediction_value > 0.5 else "Not Sarcastic"
    else:
        result = "Error: No predictions found in response."

    print(result)

Not Sarcastic
Not Sarcastic
Not Sarcastic
Not Sarcastic
Not Sarcastic
